In [17]:
import pandas as pd
import numpy as np
import urllib.parse
import time
import csv
import numpy as np
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [18]:
df = pd.read_csv('Netflix_Dataset.csv', encoding='utf-8', encoding_errors='ignore')

In [19]:
df.head()

,Show_Id,Category,Title,Director,Cast,Country,Release_Date,Rating,Duration,Type,Description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,07:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",PG-13,123 min,Dramas,A brilliant group of students become card-coun...


# Ques for TV Shows

#### Since the Duration for TV Shows is in no of seasons, which seems insufficient.
#### So Scrap the No of episodes for each of the TV shows from IMDB website

### Step1: store names of all the TV Shows first

In [20]:
tv_shows_df = df[df['Category'] != 'Movie']
tv_shows_df.head()

,Show_Id,Category,Title,Director,Cast,Country,Release_Date,Rating,Duration,Type,Description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...
11,s12,TV Show,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States","November 30, 2018",TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Dramas","In this dark alt-history thriller, a naïve law..."
12,s13,TV Show,1994,Diego Enrique Osorno,NaN,Mexico,"May 17, 2019",TV-MA,1 Season,"Crime TV Shows, Docuseries, International TV S...",Archival video and new interviews examine Mexi...
16,s17,TV Show,Feb-09,NaN,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",NaN,"March 20, 2019",TV-14,1 Season,"International TV Shows, TV Dramas","As a psychology professor faces Alzheimer's, h..."


In [21]:
tv_shows_df.to_csv('TV SHOWS_NAMES.csv', columns = ['Title'] , index = False)

### Step2: Scrap the data using the names in stored file

In [22]:
### Helper Functions to scrap the data

def search_and_get_link_for_a_show(soup_object):
    div_tag = soup_object.find('div', class_='sc-b03627f1-2 gWHDBT')
    if div_tag:
        ul_tag = div_tag.find('ul')
        if ul_tag:
            li_tag = ul_tag.find('li')
            if li_tag:
                anchor_tag = li_tag.find('a', class_='ipc-metadata-list-summary-item__t')
                if anchor_tag and 'href' in anchor_tag.attrs:
                    return 'https://www.imdb.com' + anchor_tag['href']
                
    return ''


def get_no_of_episodes(driver, link):
    if not link:
        return np.nan

    driver.get(link)
    time.sleep(2)

    soup2 = BeautifulSoup(driver.page_source, 'html.parser')
    div1 = soup2.find('div', class_='sc-57197f8e-0 jHYogi')
    if div1:
        div2 = div1.find('div', class_='sc-57197f8e-1 dSNwQI')
        if div2:
            req_div = div2.find('div', {'data-testid': 'hero-subnav-bar-left-block'})
            if req_div:
                a_tag = req_div.find('a')
                if a_tag:
                    span_tags = a_tag.find_all('span')
                    if len(span_tags) == 1:
                        return int(span_tags[0].text) if span_tags[0].text.isdigit() else np.nan
                    elif len(span_tags) == 2:
                        return int(span_tags[1].text) if span_tags[1].text.isdigit() else np.nan
                    
    return np.nan


def get_data_of_tv_shows(driver, TVShows_name):
    shows_data = []
    
    for name in TVShows_name:
        encoded_name = urllib.parse.quote(name)
        imdb_URL = f"https://www.imdb.com/find/?q={encoded_name}&s=tt&exact=true"

        driver.get(imdb_URL)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Get the show link
        link = search_and_get_link_for_a_show(soup)

        # Get number of episodes
        episodes = get_no_of_episodes(driver, link)

        shows_data.append([name, link, episodes])
    
    return shows_data

In [23]:
### Main Function

def FINAL_DATA(TV_show_data_file_path):
    all_chunks = pd.read_csv(TV_show_data_file_path, chunksize=400)

    driver = webdriver.Chrome()

    for i, chunk in enumerate(all_chunks):
        df = pd.DataFrame(chunk['Title'])
        names_of_shows = df['Title'].values
        print(f'Processing chunk {i}')

        req_shows_data = get_data_of_tv_shows(driver, names_of_shows)

        try:
            with open('tv_shows_data.csv', 'a') as file:
                csvWriter = csv.writer(file)
                csvWriter.writerow(['TV Show Name', 'IMDB Link', 'No of Episodes'])
                csvWriter.writerows(req_shows_data)
        except Exception as e:
            print(f"Error in chunk {i}: {e}")

        print(f'Chunk {i} is Done')

    driver.quit()

#### Executing The File

In [24]:
# FINAL_DATA('TV SHOWS_NAMES.csv')

### Step3: Load the Data

In [25]:
tv_shows_df = pd.read_csv('tv_shows_data.csv')

In [29]:
tv_shows_df.dropna(inplace = True)

In [30]:
names_and_episodes = tv_shows_df[['TV Show Name', 'No of Episodes']]

In [33]:
names_and_episodes.tail()

,TV Show Name,No of Episodes
2410,Zig & Sharko,133
2411,Zindagi Gulzar Hai,26
2412,Zoids Wild,30
2414,Zona Rosa,120
2415,Zumbo's Just Desserts,22
